In [1]:
%reload_ext tensorboard

import pandas as pd
import numpy as np
from matplotlib import pyplot
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
from datetime import datetime
from tensorflow import keras


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

In [3]:
# read formatted data from csv files
# Developed markets data will be used to train and validate the model
dm_index = pd.read_csv('dm_index.csv',sep=',',index_col=0)
dm_pe = pd.read_csv('dm_pe.csv',sep=',',index_col=0)
dm_pb = pd.read_csv('dm_pb.csv',sep=',',index_col=0)
dm_ps = pd.read_csv('dm_ps.csv',sep=',',index_col=0)

# Emerging markets data will be used to test the model
em_index = pd.read_csv('em_index.csv',sep=',',index_col=0)
em_pe = pd.read_csv('em_pe.csv',sep=',',index_col=0)
em_pb = pd.read_csv('em_pb.csv',sep=',',index_col=0)
em_ps = pd.read_csv('em_ps.csv',sep=',',index_col=0)

# compute one month forward returns
dm_returns = dm_index.shift(-22)/dm_index -1
em_returns = em_index.shift(-22)/em_index -1

# drop all values which are na
dm_returns = dm_returns.dropna()

# now scale all of the above
scaled_dm_returns = dm_returns.apply(lambda x:(x-x.min()) / (x.max()-x.min()),axis=1)
scaled_dm_pe = dm_pe.apply(lambda x:(x-x.min()) / (x.max()-x.min()),axis=1)
scaled_dm_pb = dm_pb.apply(lambda x:(x-x.min()) / (x.max()-x.min()),axis=1)
scaled_dm_ps = dm_ps.apply(lambda x:(x-x.min()) / (x.max()-x.min()),axis=1)

# align the dataset
scaled_dm_pe = scaled_dm_pe[scaled_dm_pe.index.isin(scaled_dm_returns.index)]
scaled_dm_pb = scaled_dm_pb[scaled_dm_pb.index.isin(scaled_dm_returns.index)]
scaled_dm_ps = scaled_dm_ps[scaled_dm_ps.index.isin(scaled_dm_returns.index)]

# create dataset where the last column is output
# split the dataset into 2/3 and 1/3 and make it into training and validation dataset
def create_dataset(df1, df2, df3, df4, colidx):
    df1 = np.array(df1.iloc[:,colidx])
    df1 = df1.reshape((len(df1),1))
    df2 = np.array(df2.iloc[:,colidx])
    df2 = df2.reshape((len(df2),1))
    df3 = np.array(df3.iloc[:,colidx])
    df3 = df3.reshape((len(df3),1))
    df4 = np.array(df4.iloc[:,colidx])
    df4 = df4.reshape((len(df4),1))
    dataset = np.hstack((df1,df2,df3,df4,df4))
    splitpnt = len(dataset) *2 //3
    train_data = dataset[:splitpnt,:]
    val_data = dataset[splitpnt+1:,:]
    
    return train_data, val_data

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [4]:
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

n_steps = 22
n_features = 4
n_seq = 1
# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(20, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [5]:
    !rm -rf ./logs/
    i = scaled_dm_returns.shape[1]-2
    t, v = create_dataset(scaled_dm_pe, scaled_dm_pb, scaled_dm_ps, scaled_dm_returns, i)
    tx, ty = split_sequences(t, n_steps)
    vx, vy = split_sequences(v, n_steps)
    tx = tx.reshape((tx.shape[0], n_seq, n_steps, n_features))
    vx = vx.reshape((vx.shape[0], n_seq, n_steps, n_features))
    filename = 'CNN-LSTM-' + scaled_dm_returns.columns[i] +'.json'
    model.fit(tx,ty,epochs=150, validation_data=(vx, vy),callbacks=[tensorboard_callback])
    history = model.fit(tx,ty,epochs=150, validation_data=(vx, vy))
    pyplot.plot(history.history['loss'])
    pyplot.plot(history.history['val_loss'])
    pyplot.title('model train vs validation loss for ' + scaled_dm_returns.columns[i])
    pyplot.ylabel('loss')
    pyplot.xlabel('epoch')
    pyplot.legend(['train', 'validation'], loc='upper right')
    pyplot.savefig(('CNN-LSTM-' + scaled_dm_returns.columns[i] + '.png'))
    pyplot.close()
    hist_df = pd.DataFrame(history.history)
    print(scaled_dm_returns.columns[i] + ' done')
    with open(filename, mode='w') as f:
        hist_df.to_json(f)
    del history
    del t
    del v
    del tx
    del ty
    del vx
    del vy
    del hist_df

Train on 3443 samples, validate on 1710 samples
Epoch 1/5
3443/3443 [==============================] - 3s 910us/sample - loss: 0.0428 - val_loss: 0.0104
Epoch 2/5
3443/3443 [==============================] - 1s 159us/sample - loss: 0.0080 - val_loss: 0.0063
Epoch 3/5
3443/3443 [==============================] - 1s 155us/sample - loss: 0.0047 - val_loss: 0.0037
Epoch 4/5
3443/3443 [==============================] - 1s 166us/sample - loss: 0.0032 - val_loss: 0.0039
Epoch 5/5
3443/3443 [==============================] - 1s 149us/sample - loss: 0.0027 - val_loss: 0.0023
Train on 3443 samples, validate on 1710 samples
Epoch 1/5
3443/3443 [==============================] - 1s 154us/sample - loss: 0.0025 - val_loss: 0.0021
Epoch 2/5
3443/3443 [==============================] - 1s 151us/sample - loss: 0.0023 - val_loss: 0.0020
Epoch 3/5
3443/3443 [==============================] - 1s 153us/sample - loss: 0.0022 - val_loss: 0.0021
Epoch 4/5
3443/3443 [==============================] - 1s 182us/

In [7]:
%tensorboard --logdir logs